In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("whiterabbit")

create_database('blp_dataset')
create_table('products_tmp_white_rabbit')

In [ ]:
folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\items\item")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                raw_desc = page_content.find("div", attrs={"id": "description"})

                if raw_desc is not None:
                    description = get_content(raw_desc)
            
                ## GET TITLE
                title = ""
                raw_title = page_content.find("h2")
                
                if raw_title is not None:
                    title = get_content(raw_title)
                
                ## GET SELLER
                seller = ""
                raw_seller = page_content.find("table", attrs={"class": "table"})

                if raw_seller is not None:
                    raw_seller = raw_seller.find("a")
                    
                    if raw_seller is not None:
                        seller = get_content(raw_seller)

                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_white_rabbit (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                    values = (title, description, "White Rabbit", seller, "t")
                    
                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    img_content = page_content.find("img", attrs={"class": "item-view-img"})
                    
                    if img_content is not None:
                        image = img_content["src"]

                        try:
                            data = "" + image.replace("data:image/jpeg;base64,", "")
                            imgdata = base64.b64decode(data)

                            img_filename = Path(f"D:\images\white_rabbit\{product_id}.jpg")

                            with open(img_filename, 'wb') as f:
                                f.write(imgdata)
                                f.close
                        except:
                            print(f"Image {product_id} has error. Except")
                            update_image(product_id, "no_image")
                    else:
                        print(f"Product {product_id} has no image")
                        update_image(product_id, "no_image")